In [2]:
# ==========================================
# 🧩 Colab All-in-One: Ollama + FastAPI + Streamlit + ngrok (✅ bản fix lỗi 403)
# ==========================================

!apt-get install -y curl > /dev/null
!pip install fastapi uvicorn nest-asyncio pyngrok streamlit requests > /dev/null

import os, json, time, subprocess, threading, requests, nest_asyncio
from pyngrok import ngrok
nest_asyncio.apply()

# ------------------------
# 1️⃣ Cài và khởi động Ollama
# ------------------------
print("🚀 Cài đặt Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

print("\n⚙️ Khởi động Ollama server...")
ollama_proc = subprocess.Popen(["ollama", "serve"])
time.sleep(10)

print("🧠 Kiểm tra model gemma:2b...")
!ollama pull gemma:2b || echo "Đã có model gemma:2b"

# ------------------------
# 2️⃣ Viết FastAPI app (gọi Ollama API nội bộ)
# ------------------------
app_code = """
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Dict
import requests, json

app = FastAPI()

# Cho phép truy cập từ bất kỳ nguồn nào (tránh lỗi CORS)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class ItineraryRequest(BaseModel):
    username: str
    origin: str
    destination: str
    dates: Dict[str, str]
    interests: List[str]
    pace: str

def generate_itinerary(prompt: str):
    url = "http://127.0.0.1:11434/api/generate"
    data = {"model": "gemma:2b", "prompt": prompt}
    try:
        response = requests.post(url, json=data, timeout=120)
        if response.status_code == 200:
            # Ghép nội dung từ các dòng streaming
            lines = [json.loads(l)["response"] for l in response.text.splitlines() if '"response"' in l]
            return "".join(lines)
        else:
            return f"Error: Ollama returned {response.status_code}: {response.text[:200]}"
    except Exception as e:
        return f"Error calling Ollama: {e}"

@app.post("/generate_itinerary")
def create_itinerary(req: ItineraryRequest):
    prompt = f'''
You are a travel planner AI. Create a day-by-day itinerary from {req.origin} to {req.destination}
from {req.dates['start']} to {req.dates['end']}.
Focus on interests: {', '.join(req.interests)}.
Use pace: {req.pace}.
Divide each day into morning, afternoon, and evening with 1–2 sentences each.
Return JSON format like:
{{"day 1": {{"morning": "...", "afternoon": "...", "evening": "..."}}}}
'''
    result = generate_itinerary(prompt)
    return {"itinerary": result}
"""

with open("app.py", "w") as f:
    f.write(app_code)
print("✅ FastAPI app.py đã được tạo.")

# ------------------------
# 3️⃣ Chạy FastAPI server (nội bộ)
# ------------------------
def run_fastapi():
    os.system("uvicorn app:app --host 0.0.0.0 --port 8000 --log-level warning")

fastapi_thread = threading.Thread(target=run_fastapi, daemon=True)
fastapi_thread.start()
time.sleep(8)
print("✅ FastAPI server đã khởi động trên localhost:8000.")

# ------------------------
# 4️⃣ Tạo Streamlit UI (frontend)
# ------------------------
streamlit_code = """
import streamlit as st
import requests
import json

st.set_page_config(page_title="Travel Itinerary Planner", page_icon="🌍", layout="centered")

st.title("🌍 Travel Itinerary Planner")
st.write("Nhập thông tin chuyến đi của bạn và nhận kế hoạch chi tiết!")

username = st.text_input("👤 Tên người dùng", "guest")
origin = st.text_input("🏙️ Điểm khởi hành", "Tokyo")
destination = st.text_input("🎯 Điểm đến", "Kyoto")
start_date = st.date_input("📅 Ngày bắt đầu")
end_date = st.date_input("📅 Ngày kết thúc")
interests = st.multiselect("💡 Sở thích", ["food", "museums", "nature", "nightlife"], ["food", "nature"])
pace = st.selectbox("⏱️ Tốc độ hành trình", ["relaxed", "normal", "tight"], index=1)

if st.button("✨ Tạo lịch trình"):
    if not username or not origin or not destination:
        st.error("⚠️ Vui lòng nhập đầy đủ thông tin.")
    else:
        payload = {
            "username": username,
            "origin": origin,
            "destination": destination,
            "dates": {
                "start": str(start_date),
                "end": str(end_date)
            },
            "interests": interests,
            "pace": pace
        }
        st.info("⏳ Đang gửi yêu cầu tới FastAPI server...")
        try:
            # Gọi trực tiếp FastAPI nội bộ
            res = requests.post("http://localhost:8000/generate_itinerary", json=payload, timeout=180)
            if res.status_code == 200:
                data = res.json()
                st.success("✅ Lịch trình đã sẵn sàng!")
                st.text_area("📜 Kết quả", data["itinerary"], height=400)
            else:
                st.error(f"❌ Lỗi: mã trả về {res.status_code}")
                st.text(res.text[:500])
        except Exception as e:
            st.error(f"🚫 Lỗi kết nối: {e}")
"""

with open("streamlit_app.py", "w") as f:
    f.write(streamlit_code)
print("✅ Đã tạo streamlit_app.py (gọi nội bộ localhost).")

# ------------------------
# 5️⃣ Chạy Streamlit và mở ngrok
# ------------------------
print("🚀 Khởi động Streamlit server...")
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "streamlit_app.py", "--server.port", "8501", "--server.address", "0.0.0.0"]
)
time.sleep(10)

# ⚠️ Thay bằng token ngrok của bạn
ngrok_token = input("Nhập Ngrok Authtoken (https://dashboard.ngrok.com/get-started/your-authtoken): ").strip()
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

streamlit_url = ngrok.connect(8501)
print(f"📺 Mở giao diện Streamlit tại: {streamlit_url.public_url}")


🚀 Cài đặt Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

⚙️ Khởi động Ollama server...
🧠 Kiểm tra model gemma:2b...

✅ FastAPI app.py đã được tạo.
✅ FastAPI server đã khởi động trên localhost:8000.
✅ Đã tạo streamlit_app.py (gọi nội bộ localhost).
🚀 Khởi động Streamlit server...
Nhập Ngrok Authtoken (https://dashboard.ngrok.com/get-started/your-authtoken): 35CVW7zt8dgrG1xqWW59CEafYfQ_59Db3VwDy2qi5HxmF2BtK
📺 Mở giao diện Streamlit tại: https://jalen-prandial-imitatively.ngrok-free.dev
